# FSNAU scraper

Web scraper script for downloading and aggregating monthly FSNAU data from the FSNAU dashboard.\
No infringment intended.\
May stop working in the future.


In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from login_data import * 
url = 'https://dashboard.fsnau.org/index/login'


In [2]:
html = urlopen(url)
soup = bs(html, 'html.parser')
s = requests.session()

# fill in password and username below
login_payload = {
    'username': USERNAME,
    'password': PASSWORD}


In [3]:
login_req = s.post(url, data=login_payload)
print(login_req.status_code == 200)


True


In [4]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']
years = range(2009, 2023)
dates = ['01-{}-{}'.format(m, y) for y in years for m in months][:-10]


In [5]:
data = []
first = False
for date in dates:
    soup = bs(s.get(
        'https://dashboard.fsnau.org/dashboard/index/{}'.format(date)).text, 'html.parser')
    table = soup.find_all('table')[0]
    rows = table.find_all('tr')
    for row in rows[1:]:
        data.append([])
        for cell in row.find_all(['td', 'th']):
            data[-1].append(cell.get_text().strip().replace(',', ''))
        data[-1].append(date)


In [6]:
df = pd.DataFrame(data, columns=[r.get_text()
                  for r in rows[0].find_all(['td', 'th'])] + ['date'])
df.to_csv('../Datasets/fsnau_full.csv', index=False)
